<h1 style='text-align: center;'>Survivor:</h1>
<h2 style='text-align: center;'>Exploring Contestant Trends Through Data</h2>

This projects aims to look at the long-running CBS show Survivor over the years, and determine what demographics of contestants are more likely to win.

We will also be looking to see if there are any trends where certain demographics have lower chances of winning.


In [1]:
import pandas as pd
import matplotlib as plt
import sqlite3
# Data pulled in/ saved to csv from 2/16/2025


## Importing Data

In [2]:
seasons = pd.read_csv('seasons.csv')
contestants = pd.read_csv('contestants.csv')
stats = pd.read_csv('stats.csv')
idols = pd.read_csv('idols.csv')
advantages = pd.read_csv('advantages.csv')
immunities = pd.read_csv('immunities.csv')

## Cleaning Data

After loading the datasets in, it's time to clean the data. 
I want to take the tables for advantages, idols, and individual immunity challenges, and add them to the stats table.

To do this, I create a function that cleans these tables. This function drops unnecessary and duplicate columns, strips column names, replaces and fills values as needed, and changes the dtypes of columns to match the values. It then merges the tables together on the Contestant and Season columns, using outer join for the advantages, idols, and immunities tables so to not lose any data, and then using a left join on the stats table, filtering out any contestants not included elsewhere in the data (ie, from seasons of the Austrailian version of the show)

In [3]:
def clean_and_merge(advantages, idols, immunities, stats):
    """
    Cleans tables for advantages, idols and immunities, then merges them to the stats table.
    """
    # drop columns
    advantages = advantages.drop(columns=['Rank', 'Contestant', 'VV', 'VFB', 'Tie broken?'])
    idols = idols.drop(columns=['Rank', 'Contestant'])
    immunities = immunities.drop(columns=['Rank', 'Contestant'])
    # change column names 
    advantages.columns = advantages.columns.str.strip().str.replace('.1', '')
    idols.columns = idols.columns.str.strip().str.replace('.1', '')
    immunities.columns = immunities.columns.str.strip().str.replace('.1', '')
    # strip and replace values (Season column of S, idols table of special characters (*,†/+,#))
    advantages['Season'] = advantages['Season'].str.replace('S', '')
    advantages['Season'] = advantages['Season'].replace({
        'Game Changers': 34,
        'David vs. Goliath': 37,
        'Winners at War': 40,
        'Cambodia': 31,
        'Island of the Idols': 39,
        'HvHvH': 35,
        'Worlds Apart': 30,
        'Kaoh Rong': 32,
        'Ghost Island': 36,
        'urvivor 42': 42,
        'Edge of Extinction': 38,
        'MvGX': 33   
    })
    advantages['Season'] = advantages['Season'].astype(int) # hopefully the other two go more smoothly

    idols['Season'] = idols['Season'].str.replace('S', '') 
    idols['Contestant'] = idols['Contestant'].str.rstrip('*').str.rstrip('#').str.rstrip('+')
    idols['IH'] = idols['IH'].str.rstrip('*').str.rstrip('#').str.rstrip('+')
    idols['IP'] = idols['IP'].str.rstrip('*').str.rstrip('#').str.rstrip('+')
    idols['VV'] = idols['VV'].str.rstrip('†').str.rstrip('#')
    idols = idols.drop(idols[idols['Season'] == '--'].index)
    idols['IH'] = idols['IH'].astype(int)
    idols['IP'] = idols['IP'].astype(int)
    idols['VV'] = idols['VV'].astype(int)
    idols['Season'] = idols['Season'].astype(int)

    immunities['Season'] = immunities['Season'].str.split(':').str[0]
    immunities['Season'] = immunities['Season'].str.strip('Survivor').str.strip('S')
    immunities['Season'] = immunities['Season'].astype(int)
    # merge advantages/idols/immunities together (before merging them to stats?)
    merged_idols = pd.merge(idols, advantages, on=['Contestant', 'Season'], how='outer')
    merged_all = pd.merge(merged_idols, immunities, on=['Contestant', 'Season'], how='outer')
    stats = pd.merge(stats, merged_all, on=['Contestant', 'Season'], how='left')
    # reorder columns to be more readable
    stats = stats[['Season', 'Contestant', 'SurvSc', 'SurvAv', 'ChW', 'ChA', 'ChW%',
                            'SO', 'VFB', 'VAP','TotV','TCA','TC%','wTCR','JVF', 'TotJ', 
                            'JV%', 'IF', 'IH', 'IP', 'VV', 'ICW', 'ICA', 'AF', 'AP', 'Notes']]
    
    stats['VAP'] = stats['VAP'].str.rstrip('*')
    stats['TotV'] = stats['TotV'].str.rstrip('*')
    stats['TCA'] = stats['TCA'].str.rstrip('*')
    stats['TC%'] = stats['TC%'].str.rstrip('*')
    stats['wTCR'] = stats['wTCR'].str.rstrip('*')
    # drop rows where SurvSc is null (Season 48 - no values for stats yet)
    stats = stats.dropna(subset='SurvSc')
    # fill null values (notes with NA, everything else with 0)
    stats['Notes'] = stats['Notes'].fillna('NA')
    stats = stats.fillna(0)
    stats = stats.replace('-', '0')
    # to float- survsc, survav, chw, cha, chw%, tc%, wtcr, jv%
    # to int- so, vfb, vap, totv, tca, jvf, totj
    stats.astype({
    'SurvSc': 'float64', 'SurvAv': 'float64', 'ChW': 'float64', 'ChA': 'float64',
    'ChW%': 'float64', 'TC%': 'float64', 'wTCR': 'float64', 'JV%': 'float64',
    'SO': 'int64', 'VFB': 'int64', 'VAP': 'int64', 'TotV': 'int64', 'TCA': 'int64',
    'JVF': 'int64', 'TotJ': 'int64'
    })
    return stats # will be used to clean idols, advantages, and immunities. 

stats = clean_and_merge(advantages, idols, immunities, stats)

stats.head()


KeyError: "['SurvSc', 'SurvAv', 'ChW', 'ChA', 'ChW%', 'SO', 'VFB', 'VAP', 'TotV', 'TCA', 'TC%', 'wTCR', 'JVF', 'TotJ', 'JV%', 'Notes'] not in index"

Next, I rename all the columns in the newly merged stats table that were abbreviated to their full meanings, to improve readability. 

In [ ]:
def rename_columns(stats) -> pd.DataFrame:
    """
    Rename columns in stats table to be more readable
    """
    stats = stats.rename(columns={
        'SurvSc': 'Survival Score',
        'SurvAv': 'Survival Average',
        'ChW': 'Challenge Wins',
        'ChA': 'Challenge Appearances',
        'ChW%': 'Challenge Win %',
        'SO': 'Sit Outs',
        'VFB': 'Votes For Bootee',
        'VAP': 'Votes Against (Total)',
        'TotV': 'Total Votes Cast',
        'TCA': 'Tribal Council Appearances',
        'TC%': 'Tribal Counicl %',
        'wTCR': 'Tribal Council Ratio (Weighted)',
        'JVF': 'Jury Votes For',
        'TotJ': 'Total Numbers Of Jurors',
        'JV%': 'Jury Votes %',
        'IF': 'Idols Found',
        'IH': 'Idols Held',
        'IP': 'Idols Played',
        'VV': 'Votes Voided',
        'ICW': 'Immunity Challenge Wins',
        'ICA': 'Immunity Challenge Appearances',
        'AF': 'Advantages Found',
        'AP': 'Advantages Played'
    })
    return stats
stats.to_csv('stats.csv', index=False)
stats = rename_columns(stats)
# update/ append or rewrite stats.csv with cleaned data (and ig the other tables (season, contestant) as well) to pull to tableau)


Next I clean the seasons table, which included renaming and reformatting column names, cleaning values in the Locations column, and changing values in the Runner-Up 2 column, where no second runner up exists.

In [ ]:
def clean_seasons(seasons) -> pd.DataFrame:
    # Rename columns
    seasons = seasons.rename(columns={
        'Runner(s)-up': 'Runner-up 1',
        'Runner(s)-up.1': 'Runner-up 2'
    })
    # Strip and format column names
    seasons.columns = seasons.columns.str.strip().str.title()
    # Clean values in Location column
    seasons['Location'] = seasons['Location'].str.split('[').str[0]
    # Replace duplicate values in Runner Up 2
    seasons.loc[seasons['Runner-Up 2'] == seasons['Runner-Up 1'], 'Runner-Up 2'] = 'NA'
    return seasons
# update csv
seasons.to_csv('seasons.csv', index=False)
seasons = clean_seasons(seasons)
seasons.head()

,Season,Subtitle,Location,Original Tribes,Winner,Runner-Up 1,Runner-Up 2,Final Vote
0,1,Borneo,"Pulau Tiga, Sabah, Malaysia",Two tribes of eight new players,Richard Hatch,Kelly Wiglesworth,NA,4–3
1,2,The Australian Outback,"Herbert River at Goshen Station, Queensland, A...",Two tribes of eight new players,Tina Wesson,Colby Donaldson,NA,4–3
2,3,Africa,"Shaba National Reserve, Kenya",Two tribes of eight new players,Ethan Zohn,Kim Johnson,NA,5–2
3,4,Marquesas,"Nuku Hiva, Marquesas Islands, French Polynesia",Two tribes of eight new players,Vecepia Towery,Neleh Dennis,NA,4–3
4,5,Thailand,"Ko Tarutao, Satun Province, Thailand",Two tribes of eight new players; picked by the...,Brian Heidik,Clay Jordan,NA,4–3


Finally, I fill the null values in the contestants table, which only consists of the Finish column for the upcoming season 48, which hasn't aired yet and has no data for this column.

In [ ]:
# fill null values
contestants['Finish'] = contestants['Finish'].fillna('TBD')
# replace Winner, Runner-up, 2nd Runner-up, Co-runner up with 1st or 2nd, respectively
contestants['Finish'] = contestants['Finish'].replace({
    'Winner': '1st',
    'Runner-up': '2nd',
    'Co-runner up': '2nd',
    '2nd Runner-up': '2nd'
})
contestants.to_csv('contestants.csv', index=False)
contestants.head()
# update csv

,Season,Name,Age,Hometown,Profession,Finish,Ethnicity,Gender,Lgbt,Has Disability
0,1,Sonja Christopher,63,"Walnut Creek, CA",Gym Teacher / Retired,16th,White,F,True,False
1,1,B.B. Andersen,64,"Mission Hills, KS",Real Estate Developer,15th,White,M,False,False
2,1,Stacey Stillman,27,"San Francisco, CA",Attorney,14th,White,F,False,False
3,1,Ramona Gray,29,"Edison, NJ",Biochemist,13th,African-American,F,False,False
4,1,Dirk Been,23,"Spring Green, WI",Dairy Farmer,12th,White,M,False,False


### Exploratory Data Analysis

In [ ]:
# how do what how does sql work
# What I want to explore:
# What are the average placements of various demographics? (Age ranges, gender, ethnicity, possibly profession and hometown)
# Are there any shared demographics of top players? (High Survivor Scores/Averages, Most Idols played, Season winners)
# What do the demographics of winners/runner ups look like? How are jury votes split amongst them and are there any potential biases present?
# What does the geographical distribution of winners look like?


In [4]:
def sql(query):
    conn = sqlite3.connect('survivor.db')

    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

conn = sqlite3.connect('survivor.db')
# Convert dataframes to SQL table
contestants.to_sql('contestants', conn, if_exists='replace', index=False)
stats.to_sql('stats', conn, if_exists='replace', index=False)
seasons.to_sql('seasons', conn, if_exists='replace', index=False)

# Close the connection
conn.close()


In [13]:
# Are there any shared demographics of top players? (High Survivor Scores/Averages, Most Idols played, Season winners)
test_sql = '''
SELECT Ethnicity, COUNT(Ethnicity) as Count
FROM contestants
WHERE Finish = '1st'
GROUP BY Ethnicity
ORDER BY COUNT(Ethnicity);
'''
sql(test_sql)

,Ethnicity,Count
0,African-Canadian,1
1,Asian-Canadian,1
2,Asian-American,3
3,African-American,4
4,Latin American,5
5,White,33


In [15]:
overall_ethnicity = '''
SELECT Ethnicity, COUNT(Ethnicity) as Count
FROM contestants
GROUP BY Ethnicity
ORDER BY Count;
'''

sql(overall_ethnicity)

,Ethnicity,Count
0,African-Canadian,3
1,Asian-Canadian,4
2,Asian-American,71
3,Latin American,71
4,African-American,127
5,White,599


In [20]:
# What are the average placements of various demographics? (Age ranges, gender, ethnicity, possibly profession and hometown)
placement_sql = '''
SELECT Gender,COUNT(Gender) as Count
FROM contestants
GROUP BY Gender
ORDER BY Count;
'''

sql(placement_sql)

,Gender,Count
0,N,2
1,F,433
2,M,440


In [ ]:

# What does the geographical distribution of winners look like?

In [ ]:
# What do the demographics of winners/runner ups look like? How are jury votes split amongst them and are there any potential biases present?